In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

In [138]:
path = './Data/trainset/trainset/Cat/'
list_images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [37]:
transform = transforms.Compose([transforms.ToTensor()])
traindata = torchvision.datasets.ImageFolder('./Data/trainset/trainset/', transform=transform)
testset =torchvision.datasets.ImageFolder('./Data/testset/', transform=transform)


size = [int(len(traindata)*0.9), len(traindata) - int(len(traindata)*0.9)]

trainset, validset = torch.utils.data.random_split(dataset=traindata, lengths=size)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=32,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(validset,
                                          batch_size=32,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=32,
                                        shuffle=True)
classes = ('Cat', 'Dog')

In [38]:
cuda_available = torch.cuda.is_available()
print (cuda_available)

True


In [39]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv = nn.Sequential(
            # Layaer 1
            nn.Conv2d(in_channels=3, out_channels=16,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 2
            nn.Conv2d(in_channels=16, out_channels=32,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 3
            nn.Conv2d(in_channels=32, out_channels=64,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
#             nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 4
            nn.Conv2d(in_channels=64, out_channels=128,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 5
            nn.Conv2d(in_channels=128, out_channels=128,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 6
            nn.Conv2d(in_channels=128, out_channels=256,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
             # Layaer 7
            nn.Conv2d(in_channels=256, out_channels=256,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layaer 8
            nn.Conv2d(in_channels=256, out_channels=512,kernel_size=(3,3) , padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
        )
        self.net = nn.Linear(512, 2)
        
    def forward(self, x):
        return self.net(self.conv(x).squeeze())

In [40]:
net = NNet()
learning_rate = 1e-3
if cuda_available:
    net.cuda()
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [41]:
c = 0
acc = []
for epoch in range(50):
    losses = []
    # Train
    for batch_idx, (images, labels) in enumerate(trainloader):
        if cuda_available:
            images, labels = images.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())

        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    net.eval()
    total = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(validloader):
        if cuda_available:
            images, targets = images.cuda(), labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        accuracy = 100.*(correct/total)
        acc.append(accuracy)

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print (lr)
    print('--------------------------------------------------------------')
    net.train()
    c += 1
    if (c%5 == 0)  and (acc[epoch] <= min(acc[epoch-4:epoch-1])):
        learning_rate *= 0.1

Epoch : 0 Loss : 0.897 
Epoch : 0 Loss : 0.658 
Epoch : 0 Loss : 0.638 
Epoch : 0 Loss : 0.629 
Epoch : 0 Loss : 0.614 
Epoch : 0 Loss : 0.601 
Epoch : 0 Loss : 0.597 
Epoch : 0 Loss : 0.588 
Epoch : 0 Loss : 0.579 
Epoch : 0 Loss : 0.573 
Epoch : 0 Loss : 0.565 
Epoch : 0 Loss : 0.561 
Epoch : 0 Test Acc : 75.000
0.001
--------------------------------------------------------------
Epoch : 1 Loss : 0.511 
Epoch : 1 Loss : 0.447 
Epoch : 1 Loss : 0.434 
Epoch : 1 Loss : 0.444 
Epoch : 1 Loss : 0.449 
Epoch : 1 Loss : 0.448 
Epoch : 1 Loss : 0.447 
Epoch : 1 Loss : 0.442 
Epoch : 1 Loss : 0.441 
Epoch : 1 Loss : 0.440 
Epoch : 1 Loss : 0.440 
Epoch : 1 Loss : 0.440 
Epoch : 1 Test Acc : 76.000
0.001
--------------------------------------------------------------
Epoch : 2 Loss : 0.433 
Epoch : 2 Loss : 0.356 
Epoch : 2 Loss : 0.348 
Epoch : 2 Loss : 0.363 
Epoch : 2 Loss : 0.365 
Epoch : 2 Loss : 0.362 
Epoch : 2 Loss : 0.365 
Epoch : 2 Loss : 0.365 
Epoch : 2 Loss : 0.368 
Epoch : 2 Loss

Epoch : 21 Loss : 0.006 
Epoch : 21 Loss : 0.007 
Epoch : 21 Loss : 0.010 
Epoch : 21 Loss : 0.012 
Epoch : 21 Loss : 0.014 
Epoch : 21 Loss : 0.015 
Epoch : 21 Loss : 0.015 
Epoch : 21 Loss : 0.014 
Epoch : 21 Loss : 0.014 
Epoch : 21 Loss : 0.013 
Epoch : 21 Loss : 0.014 
Epoch : 21 Test Acc : 84.000
0.001
--------------------------------------------------------------
Epoch : 22 Loss : 0.078 
Epoch : 22 Loss : 0.024 
Epoch : 22 Loss : 0.020 
Epoch : 22 Loss : 0.016 
Epoch : 22 Loss : 0.015 
Epoch : 22 Loss : 0.013 
Epoch : 22 Loss : 0.013 
Epoch : 22 Loss : 0.013 
Epoch : 22 Loss : 0.012 
Epoch : 22 Loss : 0.012 
Epoch : 22 Loss : 0.012 
Epoch : 22 Loss : 0.012 
Epoch : 22 Test Acc : 85.000
0.001
--------------------------------------------------------------
Epoch : 23 Loss : 0.000 
Epoch : 23 Loss : 0.011 
Epoch : 23 Loss : 0.009 
Epoch : 23 Loss : 0.007 
Epoch : 23 Loss : 0.008 
Epoch : 23 Loss : 0.007 
Epoch : 23 Loss : 0.007 
Epoch : 23 Loss : 0.008 
Epoch : 23 Loss : 0.008 
Epoc

Epoch : 41 Loss : 0.001 
Epoch : 41 Test Acc : 83.000
0.001
--------------------------------------------------------------
Epoch : 42 Loss : 0.000 
Epoch : 42 Loss : 0.000 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Loss : 0.001 
Epoch : 42 Test Acc : 86.000
0.001
--------------------------------------------------------------
Epoch : 43 Loss : 0.000 
Epoch : 43 Loss : 0.002 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Loss : 0.001 
Epoch : 43 Test Acc : 86.000
0.001
--------------------------------------------------------------
Epoch : 44 Loss : 0.000 
Epoch : 44 Loss : 0.000 
Epoch : 44 Loss : 0.000 
Epoch 

In [45]:
torch.save(net.state_dict(), './model/batchnorm.pwf')

In [47]:
# torch.save(net.state_dict(), 
#            os.path.join('saved_model', 'cnnT1_epoch_{}_iter_{}_loss_{}_acc_{}_{}.t7'.format(epoch + 1, i,
#            last_loss, acc, datetime.datetime.now().strftime("%b_%d_%H:%M:%S"))))

In [55]:
# np_image = trainset[0
# img = torch.from_numpy(np_image)
# img = img.view(img.shape[1], img.shape[2], img.shape[0])
# # img = img / 2+0.5
# # npimg = img.numpy()
# # plt.imshow(np.transpose(npimg, (1, 2, 0)))
# plt.imshow(img)
# plt.show()

In [17]:
# labels = []
# c = 0
# for (inputs, targets) in enumerate(trainloader):
#     c += 1
# print(inputs, targets)